In [ ]:
import tensorflow as tf
import pandas as pd
import re
import numpy as np
import matplotlib.pylab as plt
from sklearn.preprocessing import MinMaxScaler

In [ ]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Input, Dense, GRU, Embedding
from tensorflow.python.keras.optimizers import RMSprop
from tensorflow.python.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau

In [ ]:
path = open(r'''C:\Users\user\Documents\formatted\20162.csv''')
df = pd.read_csv(path, sep=',', header=0  , index_col=False)

In [ ]:
df.head(1)

In [ ]:
x = df[['Date', 'Time']].values
print(x)
x[3][1]

In [ ]:
def _date_string(x):
    """Convert two integers to a string for the date and time."""

    date = x[0]
    date = re.sub('-' , '' , date)
    print(date)# Date. Example: 19801231
    time = x[1]
    time = re.sub(':' , '' , time)# Time. Example: 1230
    print(time)

    return "{0}{1}".format(date , time)

In [ ]:
datestr = df[['Date', 'Time']].apply(_date_string, axis=1)
df['DateTime'] = pd.to_datetime(datestr, format='%m%d%Y%H%M')

In [ ]:
df.head(1)

In [ ]:
df['DateTime']=df['DateTime'].dt.strftime('%d-%m-%Y')

In [ ]:
datestr = df[['DateTime', 'Time']].apply(_date_string, axis=1)
df['DateTime1'] = pd.to_datetime(datestr, format='%d%m%Y%H%M')

In [ ]:
df.head(5)

In [ ]:
df=df.set_index('DateTime1')

In [ ]:
df.drop("DateTime",axis=1,inplace=True)

In [ ]:
df.head(5)

In [ ]:
df['Wind Speed'].plot()

In [ ]:
df.drop('Date' , axis = 1 , inplace = True)
df.drop('Time' , axis = 1 , inplace = True)
df.drop('Hi Dir' , axis = 1 , inplace = True)
df.drop('Wind Dir' , axis = 1 , inplace = True)


In [ ]:
df.drop('Temp Out' , axis = 1 , inplace = True)

In [ ]:
df.drop('Out Hum' , axis = 1 , inplace = True)

In [ ]:
df.drop('Dew Pt.' , axis = 1 , inplace = True)

In [ ]:
df.drop('Wind Run' , axis = 1 , inplace = True)

In [ ]:
df.drop('Hi Speed' , axis = 1 , inplace = True)

In [ ]:
df.drop('Solar Energy' , axis = 1 , inplace = True)

In [ ]:
df.drop('UV Index' , axis = 1 , inplace = True)

In [ ]:
df.drop('UV Dose' , axis = 1 , inplace = True)

In [ ]:
df.drop('Hi UV ' , axis = 1 , inplace = True)

In [ ]:
df.drop('Wind Chill' , axis = 1 , inplace = True)


In [ ]:
df.drop('Heat Index' , axis = 1 , inplace = True)


In [ ]:
df.drop('THW Index' , axis = 1 , inplace = True)

In [ ]:
df.drop('THSW Index' , axis = 1 , inplace = True)

In [ ]:
df.drop('Rain' , axis = 1 , inplace = True)

In [ ]:
df.drop('Rain Rate' , axis = 1 , inplace = True)

In [ ]:
df.drop('In Hum' , axis = 1 , inplace = True)

In [ ]:
df.drop('In Dew' , axis = 1 , inplace = True)

In [ ]:
df.drop('In Heat' , axis = 1 , inplace = True)

In [ ]:
df.drop('In EMC' , axis = 1 , inplace = True)

In [ ]:
df.drop('ET ' , axis = 1 , inplace = True)

In [ ]:
df.drop('In Air Density' , axis = 1 , inplace = True)

In [ ]:
df.drop('Wind Samp' , axis = 1 , inplace = True)

In [ ]:
df.drop('Wind Tx ' , axis = 1 , inplace = True)

In [ ]:
df.drop('ISS Recept' , axis = 1 , inplace = True)

In [ ]:
df.drop('Arc. Int.' , axis = 1 , inplace = True)

In [ ]:
df.drop('Heat D-D ' , axis = 1 , inplace = True)

In [ ]:
df.drop('Cool D-D ' , axis = 1 , inplace = True)

In [ ]:
df.dtypes

In [ ]:
df.head(5)

In [ ]:
target_names = ['Hi Temp']


In [ ]:
shift_days = 1
shift_steps = shift_days*24*60

In [ ]:
df_targets = df[target_names].shift(-shift_steps)

In [ ]:
df[target_names].head(shift_steps +5)

In [ ]:
x_data = df.values[0:-shift_steps]

In [ ]:

num_data = len(x_data)
num_data

In [ ]:
y_data = df_targets.values[:-shift_steps]

In [ ]:
train_split = 0.9


In [ ]:
num_train = int(train_split * num_data)
num_train

In [ ]:
x_train = x_data[0:num_train]
x_test = x_data[num_train:]
len(x_train) + len(x_test)

In [ ]:
y_train = y_data[0:num_train]
y_test = y_data[num_train:]
len(y_train) + len(y_test)

In [ ]:
num_x_signals = x_data.shape[1]
num_x_signals

In [ ]:
num_y_signals = y_data.shape[1]
num_y_signals

In [ ]:
print("Min:", np.min(x_train))
print("Max:", np.max(x_train))

In [ ]:
x_scaler = MinMaxScaler()

In [ ]:
x_train_scaled = x_scaler.fit_transform(x_train)

In [ ]:
print("Min:", np.min(x_train_scaled))
print("Max:", np.max(x_train_scaled))

In [ ]:
x_test_scaled = x_scaler.transform(x_test)

In [ ]:
y_scaler = MinMaxScaler()
y_train_scaled = y_scaler.fit_transform(y_train)
y_test_scaled = y_scaler.transform(y_test)

In [ ]:
print(x_train_scaled.shape)
print(y_train_scaled.shape)

In [ ]:
def batch_generator(batch_size, sequence_length):
    """
    Generator function for creating random batches of training-data.
    """

    # Infinite loop.
    while True:
        # Allocate a new array for the batch of input-signals.
        x_shape = (batch_size, sequence_length, num_x_signals)
        x_batch = np.zeros(shape=x_shape, dtype=np.float16)

        # Allocate a new array for the batch of output-signals.
        y_shape = (batch_size, sequence_length, num_y_signals)
        y_batch = np.zeros(shape=y_shape, dtype=np.float16)

        # Fill the batch with random sequences of data.
        for i in range(batch_size):
            # Get a random start-index.
            # This points somewhere into the training-data.
            idx = np.random.randint(num_train - sequence_length)
            
            # Copy the sequences of data starting at this index.
            x_batch[i] = x_train_scaled[idx:idx+sequence_length]
            y_batch[i] = y_train_scaled[idx:idx+sequence_length]
        
        yield (x_batch, y_batch)

In [ ]:
batch_size =64

In [ ]:
sequence_length = 24*7
sequence_length

In [ ]:
generator = batch_generator(batch_size=batch_size,
                            sequence_length=sequence_length)

In [ ]:
x_batch, y_batch = next(generator)

In [ ]:
print(x_batch.shape)
print(y_batch.shape)

In [ ]:
batch = 0   # First sequence in the batch.
signal = 0  # First signal from the 20 input-signals.
seq = x_batch[batch, :, signal]
plt.plot(seq)

In [ ]:
seq = y_batch[batch, :, signal]
plt.plot(seq)

In [ ]:
validation_data = (np.expand_dims(x_test_scaled, axis=0),
                   np.expand_dims(y_test_scaled, axis=0))

In [ ]:
model = Sequential()

In [ ]:
model.add(GRU(units=512,
              return_sequences=True,
              input_shape=(None, num_x_signals,)))

In [ ]:
model.add(Dense(num_y_signals, activation='sigmoid'))

In [ ]:
if False:
    from tensorflow.python.keras.initializers import RandomUniform

    # Maybe use lower init-ranges.
    init = RandomUniform(minval=-0.05, maxval=0.05)

    model.add(Dense(num_y_signals,
                    activation='linear',
                    kernel_initializer=init))

In [ ]:
warmup_steps = 50

In [ ]:
def loss_mse_warmup(y_true, y_pred):
    """
    Calculate the Mean Squared Error between y_true and y_pred,
    but ignore the beginning "warmup" part of the sequences.
    
    y_true is the desired output.
    y_pred is the model's output.
    """

    # The shape of both input tensors are:
    # [batch_size, sequence_length, num_y_signals].

    # Ignore the "warmup" parts of the sequences
    # by taking slices of the tensors.
    y_true_slice = y_true[:, warmup_steps:, :]
    y_pred_slice = y_pred[:, warmup_steps:, :]

    # These sliced tensors both have this shape:
    # [batch_size, sequence_length - warmup_steps, num_y_signals]

    # Calculate the MSE loss for each value in these tensors.
    # This outputs a 3-rank tensor of the same shape.
    loss = tf.losses.mean_squared_error(labels=y_true_slice,
                                        predictions=y_pred_slice)

    # Keras may reduce this across the first axis (the batch)
    # but the semantics are unclear, so to be sure we use
    # the loss across the entire tensor, we reduce it to a
    # single scalar with the mean function.
    loss_mean = tf.reduce_mean(loss)

    return loss_mean

In [ ]:
optimizer = RMSprop(lr=1e-4)

In [ ]:
model.compile(loss=loss_mse_warmup, optimizer=optimizer,metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
path_checkpoint = '23_checkpoint.keras'
callback_checkpoint = ModelCheckpoint(filepath=path_checkpoint,
                                      monitor='val_loss',
                                      verbose=1,
                                      save_weights_only=True,
                                      save_best_only=True)

In [ ]:
callback_early_stopping = EarlyStopping(monitor='val_loss',
                                        patience=5, verbose=1)

In [ ]:
callback_tensorboard = TensorBoard(log_dir='./23_logs/',
                                   histogram_freq=0,
                                   write_graph=False)

In [ ]:
callback_reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                                       factor=0.1,
                                       min_lr=1e-4,
                                       patience=0,
                                       verbose=1)

In [ ]:
callbacks = [callback_early_stopping,
             callback_checkpoint,
             callback_tensorboard,
             callback_reduce_lr]

In [ ]:
%%time
model.fit_generator(generator=generator,
                    epochs=20,
                    steps_per_epoch=100,
                    validation_data=validation_data,
                    callbacks=callbacks)

In [ ]:
try:
    model.load_weights(path_checkpoint)
except Exception as error:
    print("Error trying to load checkpoint.")
    print(error)

In [ ]:
result = model.evaluate(x=np.expand_dims(x_test_scaled, axis=0),
                        y=np.expand_dims(y_test_scaled, axis=0))

In [ ]:
# If you have several metrics you can use this instead.
if False:
    for res, metric in zip(result, model.metrics_names):
        print("{0}: {1:.3e}".format(metric, res))

In [ ]:
def plot_comparison(start_idx, length=100, train=True):
    """
    Plot the predicted and true output-signals.
    
    :param start_idx: Start-index for the time-series.
    :param length: Sequence-length to process and plot.
    :param train: Boolean whether to use training- or test-set.
    """
    
    if train:
        # Use training-data.
        x = x_train_scaled
        y_true = y_train
    else:
        # Use test-data.
        x = x_test_scaled
        y_true = y_test
    
    # End-index for the sequences.
    end_idx = start_idx + length
    
    # Select the sequences from the given start-index and
    # of the given length.
    x = x[start_idx:end_idx]
    y_true = y_true[start_idx:end_idx]
    
    # Input-signals for the model.
    x = np.expand_dims(x, axis=0)

    # Use the model to predict the output-signals.
    y_pred = model.predict(x)
    
    # The output of the model is between 0 and 1.
    # Do an inverse map to get it back to the scale
    # of the original data-set.
    y_pred_rescaled = y_scaler.inverse_transform(y_pred[0])
    
    # For each output-signal.
    for signal in range(len(target_names)):
        # Get the output-signal predicted by the model.
        signal_pred = y_pred_rescaled[:, signal]
        
        # Get the true output-signal from the data-set.
        signal_true = y_true[:, signal]

        # Make the plotting-canvas bigger.
        plt.figure(figsize=(15,5))
        
        # Plot and compare the two signals.
        plt.plot(signal_true, label='true')
        plt.plot(signal_pred, label='pred')
        
        # Plot grey box for warmup-period.
        p = plt.axvspan(0, warmup_steps, facecolor='black', alpha=0.15)
        
        # Plot labels etc.
        plt.ylabel(target_names[signal])
        plt.legend()
        plt.show()

In [ ]:
plot_comparison(start_idx=100000, length=1000, train=True)

In [ ]:
plot_comparison(start_idx=200000, length=1000, train=True)

In [ ]:
df['Hi Temp'][200000:200000+1000].plot()

In [ ]:
df_org = weather.load_original_data()
df_org.xs('Odense')['Temp']['2002-12-23':'2003-02-04'].plot()

In [ ]:
plot_comparison(start_idx=200, length=1000, train=False)